In [1]:
%serialconnect -p COM3

Connecting to --port=COM3 --baud=115200 
b'I (469) wifi: wifi firmware version: \xf7ets Jun  8 2016 00:22:57\r\n'
rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4732
load:0x40078000,len:7496
load:0x40080400,len:5512
entry 0x4008114c
connecting to wifi
===network found===
> auth (b0)
> assoc (0)
> run (10)

network config .... ('192.168.0.108', '255.255.255.0', '192.168.0.1', '192.168.0.1')
Traceback (most recent call last):
  File "boot.py", line 32, in <module>
  File "imu.py", line 76, in read_mpu6050
  File "imu.py", line 49, in init_MPU
OSError: [Errno 110] ETIMEDOUT
OSError: [Errno 2] ENOENT
MicroPython v1.9.4-761-g7cd59c5bc on 2018-12-24; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B

In [8]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes
    does serial.read_all()

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute]
                   [--source [SOURCE]] [--quiet] [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over USB wire

%socketconnect [--

In [4]:
import machine, ubinascii, time, math
from machine import Pin, I2C
from time import sleep
from kalman import KalmanAngle

# Default I2C address for the MPU6050
# Instantiate a kalman filter object

mpu6050_addr = 0x68
kalmanX = KalmanAngle()
kalmanY = KalmanAngle()

# Required MPU6050 registers and their addresses

PWR_MGMT_1   = 0x6B
SMPLRT_DIV   = 0x19
CONFIG       = 0x1A
GYRO_CONFIG  = 0x1B
INT_ENABLE   = 0x38
ACCEL_XOUT_H = 0x3B
ACCEL_YOUT_H = 0x3D
ACCEL_ZOUT_H = 0x3F
GYRO_XOUT_H  = 0x43
GYRO_YOUT_H  = 0x45
GYRO_ZOUT_H  = 0x47
TEMP_OUT_H   = 0X41

# Globals
last_read_time = 0.0   

# These are the filtered angles
last_x_angle = 0.0          
last_y_angle = 0.0
last_z_angle = 0.0

# Calibrated measurements to offset some bias or error in the readings.
calib_x_accel = 0.0 
calib_y_accel = 0.0 
calib_z_accel = 0.0 
calib_x_gyro  = 0.0 
calib_y_gyro  = 0.0 
calib_z_gyro  = 0.0 

#instantiate the i2c interface on esp32 (pins 21,22 for wroom32 variants) 
i2c = I2C(scl=machine.Pin(22), sda=machine.Pin(21), freq=400000)

def init_MPU():
    #write to sample rate register 
    i2c.writeto_mem(mpu6050_addr, SMPLRT_DIV, b'\x07')
    #Write to power management register to wake up mpu6050
    i2c.writeto_mem(mpu6050_addr, PWR_MGMT_1, b'\x00')
    #Write to Configuration register 
    i2c.writeto_mem(mpu6050_addr, CONFIG, b'\x00')
    #Write to Gyro configuration register to self test gyro 
    i2c.writeto_mem(mpu6050_addr, GYRO_CONFIG, b'\x18')
    #Set interrupt enable register to 0 .. disable interrupts
    i2c.writeto_mem(mpu6050_addr, INT_ENABLE, b'\x00')

def read_raw_data(addr):
    #Accelero and Gyro value are 16-bit
    high = i2c.readfrom_mem(mpu6050_addr, addr, 1)
    #print(ubinascii.hexlify(high))
    low  = i2c.readfrom_mem(mpu6050_addr, addr+1, 1)
    #print(ubinascii.hexlify(low))
    
    #concatenate higher and lower values
    val = high[0] << 8 | low[0]
        
    #we're expecting a 16 bit signed int (between -32768 to 32768). This step ensures 16 bit unsigned int raw readings are resolved. 
    if(val > 32768):
        val = val - 65536
    return val

def read_mpu6050():
    
    init_MPU()
    calibrate_sensors()
    try:
        while True:
            
            t_now = time.ticks_ms()
            dt = (t_now - get_last_time())/1000.0

            acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z = read_values_helper()

            #Full scale range +/- 250 degree/C as per sensitivity scale factor. The is linear acceleration in each of the 3 directions ins g's
            Ax = acc_x/16384.0
            Ay = acc_y/16384.0
            Az = acc_z/16384.0
            
            # This is angular velocity in each of the 3 directions 
            Gx = (gyro_x - calib_x_gyro)/131.0
            Gy = (gyro_y - calib_y_gyro)/131.0
            Gz = (gyro_z - calib_z_gyro)/131.0

            #tp = temp/ 340 + 36.53

            acc_angles = acc_angle(Ax, Ay, Az) # Calculate angle of inclination or tilt for the x and y axes with acquired acceleration vectors
            gyr_angles = gyr_angle(Gx, Gy, Gz, dt) # Calculate angle of inclination or tilt for x,y and z axes with angular rates and dt 
            (c_angle_x, c_angle_y) = c_filtered_angle(acc_angles[0], acc_angles[1], gyr_angles[0], gyr_angles[1]) # filtered tilt angle i.e. what we're after
            (k_angle_x, k_angle_y) = k_filtered_angle(acc_angles[0], acc_angles[1], Gx, Gy, dt)

            set_last_read_angles(t_now, c_angle_x, c_angle_y)
            #print ("Gx=%.6f" %Gx, u'\u00b0'+ "/s", "\tGy=%.6f" %Gy, u'\u00b0'+ "/s", "\tGz=%.6f" %Gz, u'\u00b0'+ "/s", "\tAx=%.6f g" %Ax, "\tAy=%.6f g" %Ay, "\tAz=%.6f g" %Az, "\ttemp=%.6f" %tp, u'\u00b0'+ "C") 	
    
            print("%.8f," %c_angle_x, "%.8f," %c_angle_y, "%.8f" %k_angle_x,"%.8f" %k_angle_y)

            time.sleep_ms(4)

    except KeyboardInterrupt:
        pass

def calibrate_sensors():
    x_accel = 0
    y_accel = 0
    z_accel = 0
    x_gyro  = 0
    y_gyro  = 0
    z_gyro  = 0
    
    #print("Starting Calibration")

    #Discard the first set of values read from the IMU
    read_values_helper()

    # Read and average the raw values from the IMU
    for int in range(10): 
        values = read_values_helper()
        x_accel += values[0]
        y_accel += values[1]
        z_accel += values[2]
        x_gyro  += values[3]
        y_gyro  += values[4]
        z_gyro  += values[5]
        time.sleep_ms(100)
    
    x_accel /= 10
    y_accel /= 10
    z_accel /= 10
    x_gyro  /= 10
    y_gyro  /= 10
    z_gyro  /= 10

    # Store the raw calibration values globally
    calib_x_accel = x_accel
    calib_y_accel = y_accel
    calib_z_accel = z_accel
    calib_x_gyro  = x_gyro
    calib_y_gyro  = y_gyro
    calib_z_gyro  = z_gyro

    #print("Finishing Calibration")

def set_last_read_angles(time, x, y):
    global last_read_time, last_x_angle, last_y_angle
    last_read_time = time
    last_x_angle = x 
    last_y_angle = y
    #last_z_angle = z

# accelerometer data can't be used to calculate 'yaw' angles or rotation around z axis.
def acc_angle(Ax, Ay, Az):
    radToDeg = 180/3.14159
    ax_angle = math.atan(Ay/math.sqrt(math.pow(Ax,2) + math.pow(Az, 2)))*radToDeg
    ay_angle = math.atan((-1*Ax)/math.sqrt(math.pow(Ay,2) + math.pow(Az, 2)))*radToDeg
    return (ax_angle, ay_angle)

def gyr_angle(Gx, Gy, Gz, dt):
    gx_angle = Gx*dt + get_last_x_angle()
    gy_angle = Gy*dt + get_last_y_angle()
    gz_angle = Gz*dt + get_last_z_angle()
    return (gx_angle, gy_angle, gz_angle)

  # A complementary filter to determine the change in angle by combining accelerometer and gyro values. Alpha depends on the sampling rate...
def c_filtered_angle(ax_angle, ay_angle, gx_angle, gy_angle):
    alpha = 0.90
    c_angle_x = alpha*gx_angle + (1.0 - alpha)*ax_angle
    c_angle_y = alpha*gy_angle + (1.0 - alpha)*ay_angle
    return (c_angle_x, c_angle_y)

 # Kalman filter to determine the change in angle by combining accelerometer and gyro values. 
def k_filtered_angle(ax_angle, ay_angle, Gx, Gy, dt):
    k_angle_x = kalmanX.getAngle(ax_angle, Gx, dt)
    k_angle_y = kalmanY.getAngle(ay_angle, Gy, dt)
    return (k_angle_x, k_angle_y)

def read_values_helper():

    #Read Accelerometer raw value
    acc_x = read_raw_data(ACCEL_XOUT_H)
    acc_y = read_raw_data(ACCEL_YOUT_H)
    acc_z = read_raw_data(ACCEL_ZOUT_H)
    
    #Read Gyroscope raw value
    gyro_x = read_raw_data(GYRO_XOUT_H)
    gyro_y = read_raw_data(GYRO_YOUT_H)
    gyro_z = read_raw_data(GYRO_ZOUT_H)

    #Read Temp raw value
    temp = read_raw_data(TEMP_OUT_H)

    return (acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z)

def get_last_time(): 
    return last_read_time
def get_last_x_angle():
    return last_x_angle
def get_last_y_angle():
    return last_y_angle
def get_last_z_angle():
    return last_z_angle

read_mpu6050()

.-23.67819309, 20.40627003, -4.04351616 3.13051414
-21.26557350, 18.28362465, -1.97227650 1.30900705
-19.08682251, 16.41010189, -1.16442335 0.73575182
-17.13998795, 14.76242065, -0.75937581 0.51977181
-15.38220024, 13.27555776, -0.48170662 0.36865644
-13.78100872, 12.03534007, -0.24429014 0.43931971
-12.35251904, 10.82251663, -0.07410190 0.32826972
-11.05492353, 9.73248196, 0.08816278 0.23754916
-9.91064072, 8.74522972, 0.21073124 0.13730040
-8.86607552, 7.85307884, 0.32083840 0.05428093
-7.95001888, 7.05593967, 0.37743526 -0.00893537
-7.10913849, 6.32214403, 0.45045033 -0.08970666
-6.33898973, 5.67901564, 0.52922382 -0.13006599
-5.68212795, 5.03779268, 0.54410224 -0.26401129
-5.04907084, 4.50051308, 0.63017178 -0.33055997
-4.49460793, 4.02188730, 0.67112885 -0.37305670
-3.99445677, 3.56581378, 0.70229912 -0.44443293
-3.54731774, 3.20405889, 0.71941299 -0.42891846
-3.11725330, 2.87879467, 0.77705674 -0.41578217
-2.75065804, 2.54267907, 0.79640245 -0.46526046
-2.40742445, 2.26380229, 0.

0.52898855, -0.36042566, 0.53307624 -0.37143328
0.52815695, -0.35788021, 0.53408360 -0.37230597
0.52868481, -0.36319876, 0.53548975 -0.37567601
0.53294134, -0.35945079, 0.53820877 -0.37526917
0.53585033, -0.35078244, 0.54059749 -0.37274594
0.48425899, -0.35424769, 0.52051010 -0.37506166
0.47815237, -0.35190403, 0.51752119 -0.37498214


*** Sending Ctrl-C

.48746777, -0.38782999, 0.52107944 -0.39080238


In [4]:
import imu
imu.read_mpu6050()

-0.10336764, 1.55980539, -15.20987511
0.02516862, 1.47142553, -15.20355320
0.13324488, 1.38469326, -15.19644022
0.21872206, 1.30826473, -15.18886185
0.30332878, 1.25561047, -15.18069983
0.39383469, 1.19216549, -15.17147541
0.50499291, 1.16007805, -15.16270065
0.57673550, 1.12574792, -15.15370750
0.60459971, 1.07217014, -15.14459515
0.63322268, 1.05873525, -15.13112783
0.68999090, 1.03533006, -15.12182617
0.72462869, 0.94307489, -15.11245966
0.79695964, 0.91048536, -15.10302305
0.80950060, 0.84928465, -15.09283447
0.87381611, 0.82044516, -15.08334398
0.91563587, 0.79035802, -15.07384205
0.95999804, 0.79421186, -15.06070852
0.98545065, 0.78559690, -15.05051517
1.00573039, 0.78059726, -15.04100704
0.99867516, 0.74445701, -15.03150558
1.02499068, 0.74876170, -15.02200985
1.03908336, 0.83575430, -15.01179457
1.06726432, 0.81117659, -15.00232601
1.07221508, 0.78611908, -14.99287033
1.08655953, 0.72490621, -14.98053455
1.09478247, 0.72983270, -14.97113132
1.06802201, 0.70306630, -14.96174431


1.15908515, 0.61847887, -13.17859054
1.18355989, 0.60607524, -13.16687703
1.16294515, 0.59169798, -13.15824032
1.18003130, 0.61154432, -13.15023303
1.18412436, 0.61665721, -13.14221859
1.13564396, 0.65279126, -13.13420892
1.09424186, 0.64300323, -13.12620282
1.08880782, 0.64763012, -13.11820507
1.07332599, 0.65074244, -13.10774922
1.09633493, 0.63793993, -13.09914708
1.09203410, 0.63588786, -13.09115648
1.09632802, 0.62575226, -13.08317900
1.09438205, 0.61929550, -13.07520509
1.06872725, 0.62444935, -13.06723475
1.08158517, 0.63260307, -13.05926919
1.06489372, 0.62714109, -13.05130124
1.08794594, 0.62722301, -13.03908229
1.08655500, 0.61484265, -13.03112626
1.09638929, 0.65220332, -13.02316904
1.05989063, 0.64361668, -13.01522017
1.09476673, 0.59195371, -13.00728440
1.08910275, 0.59606862, -12.99934273
1.07953703, 0.62498732, -12.99019337
1.08719277, 0.59799881, -12.98043966
1.08726883, 0.54953947, -12.97130108
1.13558412, 0.53997669, -12.96337843
1.18702078, 0.53379068, -12.95546293
1

1.31070411, 0.62485809, -11.40291810
1.33067310, 0.63152180, -11.39424562
1.37094784, 0.65991268, -11.38612628
1.36872613, 0.62170200, -11.37910724
1.35909212, 0.62244697, -11.37207985
1.34895337, 0.60210748, -11.36506677
1.34580970, 0.60858402, -11.35804534
1.31704700, 0.62691965, -11.35102272
1.31030846, 0.60629177, -11.34401321
1.31321466, 0.64854045, -11.33323312
1.30757236, 0.65224690, -11.32623911
1.29844022, 0.63223147, -11.31924748
1.30880225, 0.64620533, -11.31225467
1.26875556, 0.65299768, -11.30526423
1.25439620, 0.62126255, -11.29828334
1.23268521, 0.64508533, -11.29022956
1.23512971, 0.67399950, -11.27951741
1.20544326, 0.65565467, -11.27256155
1.21930039, 0.64607277, -11.26560807
1.18575775, 0.65322156, -11.25864863
1.19374693, 0.63130445, -11.25116348
1.16729105, 0.63713007, -11.24420524
1.14518547, 0.60267453, -11.23726487
1.18291187, 0.57954330, -11.22874141
1.18258535, 0.57513165, -11.22127533
1.20498443, 0.59796152, -11.21435523
1.17269385, 0.59052811, -11.20690227
1

1.10313249, 0.47230735, -9.60438728
1.12354100, 0.44778452, -9.59707451
1.10675645, 0.44665723, -9.58976460
1.08282709, 0.44426875, -9.58246803
1.08730769, 0.49761143, -9.57516479
1.04768264, 0.46078820, -9.56731129
1.05080485, 0.49510260, -9.55889893
1.04988551, 0.49201193, -9.54883671
1.04047143, 0.49362125, -9.54155159
1.00951064, 0.51067553, -9.53371811
1.00945330, 0.51981621, -9.52644539
0.90690031, 0.51263247, -9.51918125
0.91683245, 0.52995958, -9.51192570
0.93300428, 0.54287634, -9.50466347
0.84334793, 0.49608016, -9.49573803
0.90187559, 0.47882228, -9.48738098
0.93786449, 0.48645906, -9.48013496
0.96326265, 0.47811518, -9.47289181
0.97771797, 0.51207857, -9.46565723
0.97544584, 0.55492063, -9.45842552
0.99853401, 0.54985199, -9.45119762
1.01580513, 0.56086473, -9.44341373
1.03421402, 0.53113089, -9.43343163
1.04995990, 0.51479549, -9.42567348
1.05157137, 0.50823803, -9.41848183
1.07374954, 0.46601925, -9.41128349
1.09743667, 0.49183140, -9.40411377
1.09424532, 0.50148206, -9.3

In [1]:
!ampy -p COM3 put C:\Users\Nil\devspace\esp32\kalman.py

In [4]:
!ampy -p COM3 get imu.py

import machine, ubinascii, time, math


from machine import Pin, I2C


from time import sleep


from kalman import KalmanAngle





# Default I2C address for the MPU6050


# Instantiate a kalman filter object





mpu6050_addr = 0x68


kalmanX = KalmanAngle()


kalmanY = KalmanAngle()





# Required MPU6050 registers and their addresses





PWR_MGMT_1   = 0x6B


SMPLRT_DIV   = 0x19


CONFIG       = 0x1A


GYRO_CONFIG  = 0x1B


INT_ENABLE   = 0x38


ACCEL_XOUT_H = 0x3B


ACCEL_YOUT_H = 0x3D


ACCEL_ZOUT_H = 0x3F


GYRO_XOUT_H  = 0x43


GYRO_YOUT_H  = 0x45


GYRO_ZOUT_H  = 0x47


TEMP_OUT_H   = 0X41





# Globals


last_read_time = 0.0   





# These are the filtered angles


last_x_angle = 0.0          


last_y_angle = 0.0


last_z_angle = 0.0





# Calibrated measurements to offset some bias or error in the readings.


calib_x_accel = 0.0 


calib_y_accel = 0.0 


calib_z_accel = 0.0 


calib_x_gyro  = 0.0 


calib_y_gyro  = 0.0 


calib_z_gyro  = 0.0 





#instantiate the i2c i